In [1]:
import os
import pyodbc
import pandas as pd
import urllib
from sqlalchemy import create_engine

# --------------------------
# CONFIGURATION
# --------------------------
folder_path = r"C:\Users\Mero\Documents\hospital_data"  
server = r"localhost"           
database = "Hospital"       
batch_size = 50000  # rows per batch

# --------------------------
# TEST CONNECTION
# --------------------------
conn_str = f"Driver={{ODBC Driver 18 for SQL Server}};Server={server};Database={database};Trusted_Connection=yes;TrustServerCertificate=yes;"
try:
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    cursor.execute("SELECT GETDATE()")
    print("Connection successful! Server time:", cursor.fetchone()[0])
    conn.close()
except Exception as e:
    print("Connection failed:", e)
    exit(1)  # stop script if connection fails

# --------------------------
# CREATE SQLALCHEMY ENGINE
# --------------------------
params = urllib.parse.quote_plus(
    f"Driver={{ODBC Driver 18 for SQL Server}};"
    f"Server={server};"
    f"Database={database};"
    "Trusted_Connection=yes;"
    "TrustServerCertificate=yes;"
)
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

# --------------------------
# PROCESS ALL CSV FILES WITH LOGGING
# --------------------------
log = []  # store success/fail info

for file in os.listdir(folder_path):
    if file.endswith(".csv"):
        file_path = os.path.join(folder_path, file)
        table_name = os.path.splitext(file)[0]  # table name = file name without extension
        print(f"Loading {file} into table {table_name}...")

        try:
            chunk_iter = pd.read_csv(file_path, chunksize=batch_size)
            first_chunk = True

            for chunk in chunk_iter:
                chunk.to_sql(
                    table_name,
                    con=engine,
                    index=False,
                    if_exists='replace' if first_chunk else 'append'
                )
                first_chunk = False

            print(f"{file} loaded successfully!")
            log.append(f"{file} ✅ Success")

        except Exception as e:
            print(f"{file} failed to load. Error: {e}")
            log.append(f"{file} ❌ Failed: {e}")

# --------------------------
# SAVE LOG TO CSV
# --------------------------
log_df = pd.DataFrame(log, columns=["Status"])
log_df.to_csv(os.path.join(folder_path, "load_log.csv"), index=False)
print("All CSV files processed. Log saved to load_log.csv.")


Connection successful! Server time: 2026-02-15 05:21:47.123000
Loading admission.csv into table admission...


D:\Anaconda\Lib\site-packages\pandas\io\sql.py:1648: SAWarning: Unrecognized server version info '17.0.1050.2'.  Some SQL Server features may not function properly.
  con = self.exit_stack.enter_context(con.connect())


admission.csv loaded successfully!
Loading bed.csv into table bed...
bed.csv loaded successfully!
Loading billing.csv into table billing...
billing.csv loaded successfully!
Loading billing_detail.csv into table billing_detail...
billing_detail.csv loaded successfully!
Loading department.csv into table department...
department.csv loaded successfully!
Loading diagnostic_test.csv into table diagnostic_test...
diagnostic_test.csv loaded successfully!
Loading disease.csv into table disease...
disease.csv loaded successfully!
Loading doctor.csv into table doctor...
doctor.csv loaded successfully!
Loading drug.csv into table drug...
drug.csv loaded successfully!
Loading drug_inventory.csv into table drug_inventory...
drug_inventory.csv loaded successfully!
Loading drug_manufacturer.csv into table drug_manufacturer...
drug_manufacturer.csv loaded successfully!
Loading employee.csv into table employee...
employee.csv loaded successfully!
Loading insurance_provider.csv into table insurance_prov